## Initializing the Google Earth Engine

In [9]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=RRjiEREMHLzi2-evWt9eIwkrdNC0G0W7CWKYycFuKJI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/2QF6VF7mPyWezN5r3r9081ns7hIHz77X4kRjloOU4OtTwoItzcd7tAQ

Successfully saved authorization token.


# Image Examples

## Downloading a Dataset



In [22]:
# Get a download URL for an image.
image1 = ee.Image('CGIAR/SRTM90_V4')
path = image1.getDownloadUrl({
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': '[[-120, 35], [-119, 35], [-119, 34], [-120, 34]]'
})
print(path)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/71d18b67c93051d02931c5cf15d2b857-106f2573e8be70520c2290c6c5c01dd9:getPixels


## Display an image given a ID

In [25]:
"""Display an image given its ID."""

import ee
import ee.mapclient

image = ee.Image('CGIAR/SRTM90_V4')
ee.mapclient.addToMap(image, {'min': 0, 'max': 3000})

## HDR Landsat.
Display portions of an image with different dynamic ranges.
The land areas are displayed normally, but the water areas
are streched to show more details.


In [26]:
import datetime
import ee
import ee.mapclient

In [27]:
ee.mapclient.centerMap(-95.738, 18.453, 9)

In [28]:
# Filter the LE7 collection to a single date.
collection = (ee.ImageCollection('LANDSAT/LE07/C01/T1')
              .filterDate(datetime.datetime(2002, 11, 8),
                          datetime.datetime(2002, 11, 9)))
image = collection.mosaic().select('B3', 'B2', 'B1')

In [29]:
# Display the image normally.
ee.mapclient.addToMap(image, {'gain': '1.6, 1.4, 1.1'}, 'Land')


In [30]:
# Add and stretch the water.  Once where the elevation is masked,
# and again where the elevation is zero.
elev = ee.Image('CGIAR/SRTM90_V4')

In [31]:
mask1 = elev.mask().eq(0).And(image.mask())
mask2 = elev.eq(0).And(image.mask())

In [32]:
ee.mapclient.addToMap(
    image.mask(mask1), {'gain': 6.0, 'bias': -200}, 'Water: Masked')
ee.mapclient.addToMap(
    image.mask(mask2), {'gain': 6.0, 'bias': -200}, 'Water: Elev 0')

## Compute hillshade from elevation

In [33]:
import math
import ee
import ee.mapclient

In [34]:
ee.mapclient.centerMap(-121.767, 46.852, 11)

In [35]:
def Radians(img):
  return img.toFloat().multiply(math.pi).divide(180)


In [36]:
def Hillshade(az, ze, slope, aspect):
  """Compute hillshade for the given illumination az, el."""
  azimuth = Radians(ee.Image(az))
  zenith = Radians(ee.Image(ze))
  # Hillshade = cos(Azimuth - Aspect) * sin(Slope) * sin(Zenith) +
  #     cos(Zenith) * cos(Slope)
  return (azimuth.subtract(aspect).cos()
          .multiply(slope.sin())
          .multiply(zenith.sin())
          .add(
              zenith.cos().multiply(slope.cos())))


In [37]:
terrain = ee.Algorithms.Terrain(ee.Image('CGIAR/SRTM90_V4'))
slope_img = Radians(terrain.select('slope'))
aspect_img = Radians(terrain.select('aspect'))

In [38]:
# Add 1 hillshade at az=0, el=60.
ee.mapclient.addToMap(Hillshade(0, 60, slope_img, aspect_img))

## HSV-based Pan-Sharpening example

In [39]:
# There are many fine places to look here is one.  Comment
# this out if you want to twiddle knobs while panning around.
ee.mapclient.centerMap(-61.61625, -11.64273, 14)


In [40]:
# Grab a sample L7 image and pull out the RGB and pan bands
# in the range (0, 1).  (The range of the pan band values was
# chosen to roughly match the other bands.)
image1 = ee.Image('LANDSAT/LE07/C01/T1/LE07_230068_19990815')


In [41]:
rgb = image1.select('B3', 'B2', 'B1').unitScale(0, 255)
gray = image1.select('B8').unitScale(0, 155)

In [42]:
# Convert to HSV, swap in the pan band, and convert back to RGB.
huesat = rgb.rgbToHsv().select('hue', 'saturation')
upres = ee.Image.cat(huesat, gray).hsvToRgb()


In [43]:
# Display before and after layers using the same vis parameters.
visparams = {'min': [.15, .15, .25], 'max': [1, .9, .9], 'gamma': 1.6}
ee.mapclient.addToMap(rgb, visparams, 'Orignal')
ee.mapclient.addToMap(upres, visparams, 'Pansharpened')

## Landcover cleanup

Display the MODIS land cover classification image with appropriate colors

In [44]:
ee.mapclient.centerMap(-113.41842, 40.055489, 6)

In [45]:
# Force projection of 500 meters/pixel, which is the native MODIS resolution.
VECTORIZATION_SCALE = 500

image1 = ee.Image('MODIS/051/MCD12Q1/2001_01_01')
image2 = image1.select(['Land_Cover_Type_1'])
image3 = image2.reproject('EPSG:4326', None, 500)
image4 = image3.focal_mode()
image5 = image4.focal_max(3).focal_min(5).focal_max(3)
image6 = image5.reproject('EPSG:4326', None, 500)

In [46]:
PALETTE = [
    'aec3d4',  # water
    '152106', '225129', '369b47', '30eb5b', '387242',  # forest
    '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # shrub, grass, savannah
    '111149',  # wetlands
    'cdb33b',  # croplands
    'cc0013',  # urban
    '33280d',  # crop mosaic
    'd7cdcc',  # snow and ice
    'f7e084',  # barren
    '6f6f6f'   # tundra
    ]

In [47]:
vis_params = {'min': 0, 'max': 17, 'palette': PALETTE}

ee.mapclient.addToMap(image2, vis_params, 'IGBP classification')
ee.mapclient.addToMap(image3, vis_params, 'Reprojected')
ee.mapclient.addToMap(image4, vis_params, 'Mode')
ee.mapclient.addToMap(image5, vis_params, 'Smooth')
ee.mapclient.addToMap(image6, vis_params, 'Smooth')

## Example of *Where* operator

Select the forest classes from the MODIS land cover image and intersect them
with elevations above 1000m.

In [48]:
ee.mapclient.centerMap(-113.41842, 40.055489, 6)

In [49]:
elev = ee.Image('CGIAR/SRTM90_V4')
cover = ee.Image('MODIS/051/MCD12Q1/2001_01_01').select('Land_Cover_Type_1')
blank = ee.Image(0)


In [50]:
# Where (1 <= cover <= 4) and (elev > 1000), set the output to 1.
output = blank.where(
    cover.lte(4).And(cover.gte(1)).And(elev.gt(1000)),
    1)

In [53]:
# Output contains 0s and 1s.  Mask it with itself to get rid of the 0s.
result = output.mask(output)

ee.mapclient.addToMap(result, {'palette': '00AA00'})

# ImageCollection

## Composite an image collection and clip it to a boundary from a fusion table

See also: Filtered Seasonal Composite, which filters the
collection by bounds instead

In [54]:
import datetime

In [55]:
ee.mapclient.centerMap(-120, 37, 6)

In [56]:
# Create a Landsat 7, median-pixel composite for Spring of 2000.
collection = (ee.ImageCollection('LANDSAT/LE07/C01/T1')
              .filterDate(datetime.datetime(2000, 4, 1),
                          datetime.datetime(2000, 7, 1)))
image1 = collection.median()

In [57]:
# Clip to the output image to the California state boundary.
fc = (ee.FeatureCollection('TIGER/2018/States')
      .filter(ee.Filter().eq('NAME', 'California')))
image2 = image1.clipToCollection(fc)

In [58]:
# Select the red, green and blue bands.
image = image2.select('B3', 'B2', 'B1')
ee.mapclient.addToMap(image, {'gain': [1.4, 1.4, 1.1]})

## Map an expression

Computes the mean NDVI and SAVI by mapping an expression over a collection
and taking the mean.  This intentionally exercises both variants of
Image.expression.

In [59]:
# Filter the L7 collection to a single month.
collection = (ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA')
              .filterDate(datetime.datetime(2002, 11, 1),
                          datetime.datetime(2002, 12, 1)))


In [60]:
def NDVI(image):
  """A function to compute NDVI."""
  return image.expression('float(b("B4") - b("B3")) / (b("B4") + b("B3"))')


In [61]:
def SAVI(image):
  """A function to compute Soil Adjusted Vegetation Index."""
  return ee.Image(0).expression(
      '(1 + L) * float(nir - red)/ (nir + red + L)',
      {
          'nir': image.select('B4'),
          'red': image.select('B3'),
          'L': 0.2
      })


In [62]:
vis = {
    'min': 0,
    'max': 1,
    'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163',
        '99B718', '74A901', '66A000', '529400', '3E8601',
        '207401', '056201', '004C00', '023B01', '012E01',
        '011D01', '011301'
    ]}

In [63]:
ee.mapclient.centerMap(-93.7848, 30.3252, 11)
ee.mapclient.addToMap(collection.map(NDVI).mean(), vis)
ee.mapclient.addToMap(collection.map(SAVI).mean(), vis)

## Filter an image collection by date and region to make a median composite

See also: Clipped composite, which crops the output image
instead of filtering the input collection.

In [64]:
ee.mapclient.centerMap(-110, 40, 5)

In [65]:
# Filter to only include images within the colorado and utah boundaries.
polygon = ee.Geometry.Polygon([[
    [-109.05, 37.0], [-102.05, 37.0], [-102.05, 41.0],   # colorado
    [-109.05, 41.0], [-111.05, 41.0], [-111.05, 42.0],   # utah
    [-114.05, 42.0], [-114.05, 37.0], [-109.05, 37.0]]])


In [66]:
# Create a Landsat 7 composite for Spring of 2000, and filter by
# the bounds of the FeatureCollection.
collection = (ee.ImageCollection('LANDSAT/LE07/C01/T1')
              .filterDate(datetime.datetime(2000, 4, 1),
                          datetime.datetime(2000, 7, 1))
              .filterBounds(polygon))

In [67]:
# Select the median pixel.
image1 = collection.median()


In [68]:
# Select the red, green and blue bands.
image = image1.select('B3', 'B2', 'B1')
ee.mapclient.addToMap(image, {'gain': [1.4, 1.4, 1.1]})

# FeatureCollection

## Buffer Example 

Display the area within 2 kilometers of any San Francisco BART station.

In [69]:
ee.mapclient.centerMap(-122.4, 37.7, 11)

In [70]:

bart_stations = ee.FeatureCollection('GOOGLE/EE/DEMOS/bart-locations')
buffered = bart_stations.map(lambda f: f.buffer(2000))
unioned = buffered.union()

ee.mapclient.addToMap(unioned, {'color': '800080'})

## Computed area filter example.

Find US counties smaller than 3k square kilometers in area.

In [71]:
ee.mapclient.centerMap(-119.7, 38.26, 7)

counties = ee.FeatureCollection('TIGER/2018/Counties')
counties_with_area = counties.map(lambda f: f.set({'area': f.area()}))
small_counties = counties_with_area.filterMetadata('area', 'less_than', 3e9)

ee.mapclient.addToMap(small_counties, {'color': '900000'})

## Count feature example

Count Panoramio photos near SF that mention bridges.

In [72]:
from __future__ import print_function

In [73]:
ee.mapclient.centerMap(-122.39, 37.7857, 12)

photos_near_sf = ee.FeatureCollection('GOOGLE/EE/DEMOS/sf-photo-locations')

In [74]:
bridge_photos = photos_near_sf.filter(
    ee.Filter().Or(ee.Filter.stringContains('title', 'Bridge'),
                   ee.Filter.stringContains('title', 'bridge')))

In [75]:
ee.mapclient.addToMap(photos_near_sf, {'color': '0040b0'})
ee.mapclient.addToMap(bridge_photos, {'color': 'e02070'})

In [76]:
print('There are {} bridge photos around SF.'
      .format(bridge_photos.size().getInfo()))

There are 168 bridge photos around SF.


## Create and render a feature collection from polygons

In [77]:
ee.mapclient.centerMap(-107, 41, 6)

fc = ee.FeatureCollection([
    ee.Feature(
        ee.Geometry.Polygon(
            [[-109.05, 41], [-109.05, 37], [-102.05, 37], [-102.05, 41]]),
        {'name': 'Colorado', 'fill': 1}),
    ee.Feature(
        ee.Geometry.Polygon(
            [[-114.05, 37.0], [-109.05, 37.0], [-109.05, 41.0],
             [-111.05, 41.0], [-111.05, 42.0], [-114.05, 42.0]]),
        {'name': 'Utah', 'fill': 2})
    ])


In [78]:
# Fill, then outline the polygons into a blank image.
image1 = ee.Image(0).mask(0)
image2 = image1.paint(fc, 'fill')  # Get color from property named 'fill'
image3 = image2.paint(fc, 3, 5)    # Outline using color 3, width 5.
image4 = image3.toByte()

In [79]:
ee.mapclient.addToMap(image4, {
    'palette': ['000000', 'FF0000', '00FF00', '0000FF'],
    'max': 3,
    'opacity': 0.5
})

## FeatureCollection Join example.

Show parks in San Francisco within 2 kilometers of a BART station.

In [80]:
ee.mapclient.centerMap(-122.45, 37.75, 13)

In [81]:

bart = ee.FeatureCollection('GOOGLE/EE/DEMOS/bart-locations')
parks = ee.FeatureCollection('GOOGLE/EE/DEMOS/sf-parks')
buffered_bart = bart.map(lambda f: f.buffer(2000))


In [82]:
join_filter = ee.Filter.withinDistance(2000, '.geo', None, '.geo')
close_parks = ee.Join.simple().apply(parks, bart, join_filter)


In [83]:
ee.mapclient.addToMap(buffered_bart, {'color': 'b0b0b0'})
ee.mapclient.addToMap(close_parks, {'color': '008000'})

## Reverse mask a region

Create an image that masks everything except for the specified polygon.

In [84]:
ee.mapclient.centerMap(-115, 39, 5)

In [85]:
fc = (ee.FeatureCollection('RESOLVE/ECOREGIONS/2017')
      .filter(ee.Filter().eq('ECO_NAME', 'Great Basin shrub steppe')))

In [86]:
# Start with a black image.
empty = ee.Image(0).toByte()
# Fill and outline the polygons in two colors
filled = empty.paint(fc, 2)
both = filled.paint(fc, 1, 5)
# Mask off everything that matches the fill color.
result = both.mask(filled.neq(2))

In [87]:
ee.mapclient.addToMap(result, {
    'palette': '000000,FF0000',
    'max': 1,
    'opacity': 0.5
})